<a href="https://colab.research.google.com/github/LeechXDD/9417_Pro_Project/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

##Data preprocessing

In [17]:
dtypes={
    'elapsed_time':np.int32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    'room_coor_x':np.float32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'text':'category',
    'fqid':'category',
    'room_fqid':'category',
    'text_fqid':'category',
    'fullscreen':'category',
    'hq':'category',
    'music':'category',
    'level_group':'category'}

In [18]:
from google.colab import drive
drive.mount('/content/drive')

dataset_df = pd.read_csv('/content/drive/MyDrive/9417project/predict-student-performance-from-game-play.zip (Unzipped Files)/train.csv', dtype=dtypes)
labels = pd.read_csv('/content/drive/MyDrive/9417project/predict-student-performance-from-game-play.zip (Unzipped Files)/train_labels.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

In [20]:
CATEGORICAL = ['event_name', 'name','fqid', 'room_fqid', 'text_fqid']
NUMERICAL = ['elapsed_time','level','page','room_coor_x', 'room_coor_y',
        'screen_coor_x', 'screen_coor_y', 'hover_duration']

In [21]:
# Reference: https://www.kaggle.com/code/cdeotte/random-forest-baseline-0-664/notebook

def feature_engineer(dataset_df):
    dfs = []
    for c in CATEGORICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in NUMERICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('mean')
        dfs.append(tmp)
    for c in NUMERICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)
    dataset_df = pd.concat(dfs,axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id')
    return dataset_df

In [22]:
dataset_df = feature_engineer(dataset_df)

In [23]:
def split_dataset(dataset, test_ratio=0.20):
    USER_LIST = dataset.index.unique()
    split = int(len(USER_LIST) * (1 - 0.20))
    return dataset.loc[USER_LIST[:split]], dataset.loc[USER_LIST[split:]]

train_x, valid_x = split_dataset(dataset_df)

In [24]:
# Fetch the unique list of user sessions in the validation dataset. We assigned
# `session_id` as the index of our feature engineered dataset. Hence fetching
# the unique values in the index column will give us a list of users in the
# validation set.
VALID_USER_LIST = valid_x.index.unique()

# Create a dataframe for storing the predictions of each question for all users
# in the validation set.
# For this, the required size of the data frame is:
# (no: of users in validation set  x no of questions).
# We will initialize all the predicted values in the data frame to zero.
# The dataframe's index column is the user `session_id`s.
prediction_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)

# Create an empty dictionary to store the models created for each question.
models = {}

f1_scores = []

# Create an empty dictionary to store the evaluation score for each question.
evaluation_dict ={}

In [25]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('label')  # Assume 'label' is your target column
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [35]:
def prepare_data(df, sequence_length):
    """
    Prepare data for LSTM, based on sequence length.
    """
    # Get labels
    labels = df.groupby(df.index)["correct"].last().values
    # Drop the 'correct' and 'level_group' columns from the features
    features = df.drop(columns=["correct", "level_group"])

    data = []
    target = []

    # Create sequences
    for user, group in features.groupby(features.index):
        user_features = group.values
        if len(user_features) < sequence_length:
            padding = np.zeros((sequence_length - len(user_features), user_features.shape[1]))
            user_features = np.concatenate((user_features, padding))
        data.append(user_features)
        target.append(labels[len(data)-1])

    # Convert lists to numpy arrays
    data = np.array(data)
    target = np.array(target)

    return data, target


In [36]:
# model define
input_shape = (100, 32)

In [37]:
sequence_length = 100
# Iterate through questions 1 to 18 to train models for each question, evaluate
# the trained model and store the predicted values.
for q_no in range(1,19):
    # Select level group for the question based on the q_no.
    if q_no<=3: grp = '0-4'
    elif q_no<=13: grp = '5-12'
    elif q_no<=22: grp = '13-22'
    print("### q_no", q_no, "grp", grp)

    # Filter the rows in the datasets based on the selected level group.
    train_df = train_x.loc[train_x.level_group == grp]
    train_users = train_df.index.values
    valid_df = valid_x.loc[valid_x.level_group == grp]
    valid_users = valid_df.index.values

    print(len(valid_users))

    # Select the labels for the related q_no.
    train_labels = labels.loc[labels.q==q_no].set_index('session').loc[train_users]
    valid_labels = labels.loc[labels.q==q_no].set_index('session').loc[valid_users]

    # Add the label to the filtered datasets.
    train_df["correct"] = train_labels["correct"]
    valid_df["correct"] = valid_labels["correct"]

    # Prepare data for LSTM
    train_data, train_labels = prepare_data(train_df, sequence_length)
    valid_data, valid_labels = prepare_data(valid_df, sequence_length)




    print(len(valid_data))
    print(len(valid_labels))

    # Define the model
    model = Sequential()
    model.add(LSTM(64, input_shape=(sequence_length, train_data.shape[-1]), return_sequences=True))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(train_data, train_labels, epochs=3, validation_data=(valid_data, valid_labels))

    # Store the model
    models[f'{grp}_{q_no}'] = model

    # Evaluate the model
    evaluation = model.evaluate(valid_data, valid_labels)
    evaluation_dict[q_no] = evaluation[1]
    print(f"Validation accuracy: {evaluation[1]}")

    # Make predictions
    predictions = model.predict(valid_data)
    prediction_df.loc[valid_users, q_no-1] = predictions.flatten()


    binary_predictions = [1 if p > 0.5 else 0 for p in predictions]
    # Get the true labels
    true_labels = valid_labels[:len(valid_data)]

    # Calculate the F1 score and add it to the list
    f1 = f1_score(true_labels, binary_predictions)
    f1_scores.append(f1)



### q_no 1 grp 0-4
4713


<ipython-input-37-e403af42e414>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["correct"] = train_labels["correct"]
<ipython-input-37-e403af42e414>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df["correct"] = valid_labels["correct"]


4713
4713
Epoch 1/3
590/590 [==============================] - 84s 134ms/step - loss: 0.5902 - accuracy: 0.7250 - val_loss: 0.5813 - val_accuracy: 0.7322
Epoch 2/3
590/590 [==============================] - 72s 121ms/step - loss: 0.5876 - accuracy: 0.7263 - val_loss: 0.5812 - val_accuracy: 0.7322
Epoch 3/3
148/148 [==============================] - 5s 35ms/step - loss: 0.5830 - accuracy: 0.7322
Validation accuracy: 0.732230007648468
148/148 [==============================] - 6s 34ms/step
### q_no 2 grp 0-4
4713


<ipython-input-37-e403af42e414>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["correct"] = train_labels["correct"]
<ipython-input-37-e403af42e414>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df["correct"] = valid_labels["correct"]


4713
4713
Epoch 1/3
590/590 [==============================] - 75s 122ms/step - loss: 0.1109 - accuracy: 0.9782 - val_loss: 0.1159 - val_accuracy: 0.9756
Epoch 2/3
590/590 [==============================] - 71s 121ms/step - loss: 0.0999 - accuracy: 0.9796 - val_loss: 0.1165 - val_accuracy: 0.9756
Epoch 3/3
148/148 [==============================] - 5s 36ms/step - loss: 0.1181 - accuracy: 0.9756
Validation accuracy: 0.9755994081497192
148/148 [==============================] - 6s 34ms/step
### q_no 3 grp 0-4
4713


...

4713
4713
Epoch 1/3
590/590 [==============================] - 48s 76ms/step - loss: 0.5857 - accuracy: 0.7305 - val_loss: 0.5637 - val_accuracy: 0.7494
Epoch 2/3
590/590 [==============================] - 44s 75ms/step - loss: 0.5823 - accuracy: 0.7312 - val_loss: 0.5654 - val_accuracy: 0.7494
Epoch 3/3
148/148 [==============================] - 3s 18ms/step - loss: 0.5640 - accuracy: 0.7494
Validation accuracy: 0.7494165301322937
148/148 [==============================] - 3s 19ms/step
### q_no 17 grp 13-22
4713


<ipython-input-37-e403af42e414>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["correct"] = train_labels["correct"]
<ipython-input-37-e403af42e414>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df["correct"] = valid_labels["correct"]


4713
4713
Epoch 1/3
590/590 [==============================] - 50s 80ms/step - loss: 0.6267 - accuracy: 0.6832 - val_loss: 0.6078 - val_accuracy: 0.7036
Epoch 2/3
590/590 [==============================] - 44s 75ms/step - loss: 0.6247 - accuracy: 0.6839 - val_loss: 0.6105 - val_accuracy: 0.7036
Epoch 3/3
148/148 [==============================] - 3s 18ms/step - loss: 0.6092 - accuracy: 0.7036
Validation accuracy: 0.7035858035087585
148/148 [==============================] - 3s 18ms/step
### q_no 18 grp 13-22
4713


<ipython-input-37-e403af42e414>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["correct"] = train_labels["correct"]
<ipython-input-37-e403af42e414>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df["correct"] = valid_labels["correct"]


4713
4713
Epoch 1/3
590/590 [==============================] - 49s 77ms/step - loss: 0.2068 - accuracy: 0.9488 - val_loss: 0.1959 - val_accuracy: 0.9516
Epoch 2/3
590/590 [==============================] - 44s 75ms/step - loss: 0.1982 - accuracy: 0.9504 - val_loss: 0.1939 - val_accuracy: 0.9516
Epoch 3/3
148/148 [==============================] - 3s 18ms/step - loss: 0.1946 - accuracy: 0.9516
Validation accuracy: 0.9516231417655945
148/148 [==============================] - 3s 19ms/step


In [38]:
for name, value in evaluation_dict.items():
  print(f"question {name}: accuracy {value:.4f}")

print("\nAverage accuracy", sum(evaluation_dict.values())/18)

question 1: accuracy 0.7322
question 2: accuracy 0.9756
question 3: accuracy 0.9351
question 4: accuracy 0.7919
question 5: accuracy 0.5557
question 6: accuracy 0.7872
question 7: accuracy 0.7460
question 8: accuracy 0.6331
question 9: accuracy 0.7583
question 10: accuracy 0.4870
question 11: accuracy 0.6505
question 12: accuracy 0.8701
question 13: accuracy 0.7161
question 14: accuracy 0.7297
question 15: accuracy 0.5103
question 16: accuracy 0.7494
question 17: accuracy 0.7036
question 18: accuracy 0.9516

Average accuracy 0.737970620393753


In [42]:
# Print the F1 score for each question
for i, score in enumerate(f1_scores, start=1):
    print(f"Question {i} : F1 {score:.4f}")

# Calculate and print the average F1 score
average_f1 = sum(f1_scores) / len(f1_scores)
print(f"\nAverage F1 score: {average_f1:.4f}")

Question 1 : F1 0.8454
Question 2 : F1 0.9876
Question 3 : F1 0.9664
Question 4 : F1 0.8838
Question 5 : F1 0.7144
Question 6 : F1 0.8809
Question 7 : F1 0.8545
Question 8 : F1 0.7754
Question 9 : F1 0.8626
Question 10 : F1 0.0000
Question 11 : F1 0.7883
Question 12 : F1 0.9306
Question 13 : F1 0.0000
Question 14 : F1 0.8437
Question 15 : F1 0.0000
Question 16 : F1 0.8568
Question 17 : F1 0.8260
Question 18 : F1 0.9752

Average F1 score: 0.7218
